In [1]:
import pandas as pd
from pandas.io import gbq
import pickle
import json

import time
import datetime
import scipy.sparse as sparse

from pitched_recommend import Recommender

In [2]:
with open('model_20181112_lighter.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
len(model.isrcs)

1872604

In [2]:
with open('/Users/daria/Downloads/recommender_model_20181112.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
classics = pd.read_csv('/Users/daria/Downloads/https___pitched-optimize.umusic.net_exports_dmescherina_20190211113005.csv')
classics = classics.drop(columns=['Name','Description','Image','Label','Duration','Upc','AddedAt'])

col_dict = {
    'PlaylistUri':'playlist_uri',
    'Isrc':'isrc',
    'Artist':'track_artist',
    'TrackName':'track_title',
    'AddedAt':'date_add'
}

classics = classics.rename(columns=col_dict)
classics.head()

,playlist_uri,isrc,track_artist,track_title
0,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USUM70953591,Blue October,Blue Does
1,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USCA21002636,Good Charlotte,Cardiology
2,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USIR10500509,Audioslave,Be Yourself
3,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USAM19400007,Soundgarden,Black Hole Sun
4,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USCJY1423011,Aaron Lewis,Afraid


In [4]:
classics['days_on_playlist'] = 30

In [5]:
classics.shape

(22, 5)

In [6]:
classics.head()

,playlist_uri,isrc,track_artist,track_title,days_on_playlist
0,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USUM70953591,Blue October,Blue Does,30
1,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USCA21002636,Good Charlotte,Cardiology,30
2,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USIR10500509,Audioslave,Be Yourself,30
3,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USAM19400007,Soundgarden,Black Hole Sun,30
4,spotify:user:dmescherina:playlist:4O7qbIrecBZC...,USCJY1423011,Aaron Lewis,Afraid,30


In [7]:
classics = classics[classics.isrc.isin(model.isrcs)]
classics = classics.reset_index(drop=True)
classics.shape

(14, 5)

In [8]:
model.isrcs[:5]

['TCACT1624704',
 'USUM71318966',
 'USMC10400640',
 'NLS5K1700040',
 'ARG511000031']

In [10]:
len(model.isrcs)

1872604

In [9]:
model.isrc_lookup.head()

,isrc,track_artist,track_title
0,TCACT1624704,Kuselofte,Junkyard 2017
1,USUM71318966,St. Vincent,Digital Witness
2,USMC10400640,Snoop Dogg,Drop It Like It's Hot
3,NLS5K1700040,Jairzinho,Matta
4,ARG511000031,Eiza González,Contigo Todo


In [17]:
len(model.playlists)

180866

In [11]:
model.isrc_lookup.shape

(1996622, 3)

In [12]:
model.isrc_lookup.nunique()

isrc            1872604
track_artist     290720
track_title     1337667
dtype: int64

In [13]:
dedup = model.isrc_lookup.drop_duplicates(subset=['isrc']).reset_index(drop=True)
dedup.head()

,isrc,track_artist,track_title
0,TCACT1624704,Kuselofte,Junkyard 2017
1,USUM71318966,St. Vincent,Digital Witness
2,USMC10400640,Snoop Dogg,Drop It Like It's Hot
3,NLS5K1700040,Jairzinho,Matta
4,ARG511000031,Eiza González,Contigo Todo


In [54]:
dedup1 = model.isrc_lookup.drop_duplicates(subset=['isrc']).reset_index(drop=True)

In [56]:
gbq.to_gbq(dedup1, 'recommender.track_metadata', if_exists='replace', 
           project_id='umg-comm-tech-dev')

1it [00:05,  5.82s/it]


In [17]:
lookup = dedup.set_index('isrc')
lookup_dict = lookup.to_dict()

In [18]:
lookup_dict

{'track_artist': {'TCACT1624704': 'Kuselofte',
  'USUM71318966': 'St. Vincent',
  'USMC10400640': 'Snoop Dogg',
  'NLS5K1700040': 'Jairzinho',
  'ARG511000031': 'Eiza González',
  'USUM71303140': 'Florida Georgia Line',
  'ITB008400549': 'Rino Gaetano',
  'JPVP01107802': 'Yoshihisa Hirano',
  'NLE871237001': 'Gebroeders Ko',
  'USCA29601162': 'Poison',
  'GB2LD0902040': 'Jim Yosef',
  'DEB581505963': 'Benjamin Blümchen',
  'TCACG1528047': 'Jay Rock',
  'DEEQ81800041': 'Peter Wackel',
  'GBBKS0800398': 'Friendly Fires',
  'USIR10211066': 'Eminem',
  'USSD19200270': 'Rey Ruiz',
  'GBFFP0300054': 'The Killers',
  'USUYG1123864': 'Fat Joe',
  'GBAYE9400061': 'Radiohead',
  'BXPYA1800053': 'Mc Leléto',
  'MXUM71502000': 'Mon Laferte',
  'ES5151502835': 'Pablo Alborán',
  'USUM71702488': 'Katy Perry',
  'USUM71805289': 'Ariana Grande',
  'USCM51400389': 'Nicki Minaj',
  'DEKC41500115': 'Neelix',
  'USLD91700854': 'blackbear',
  'NLS5K1700079': 'Sevn Alias',
  'USASN1401256': 'Los Gfez',
  'D

In [14]:
start_time = time.time()
classic_recs = model.recommend('spotify:user:musicologo13:playlist:0000mSEZofZjMa5x6ooFMT', 
                               model.playlists_sparse, N=300)
rec_time = time.time() - start_time
print(rec_time)
df_recs = model.rec_readable(classic_recs)
all_time = time.time() -start_time
print(all_time)

0.31675219535827637
96.12724804878235


In [16]:
df_recs.head(20)

,scores,isrc,artist,title,seed_playlist_uri
0,1.200714,USQX91100088,The Ronettes,Be My Baby,spotify:user:musicologo13:playlist:0000mSEZofZ...
1,1.198107,GB01A0800374,Del Shannon,Runaway,spotify:user:musicologo13:playlist:0000mSEZofZ...
2,1.194081,USLA19100004,Dion,Runaround Sue - 1991 - Remaster,spotify:user:musicologo13:playlist:0000mSEZofZ...
3,1.186085,GBAYE6200144,Little Eva,The Locomotion,spotify:user:musicologo13:playlist:0000mSEZofZ...
4,1.172477,USMO16490001,The Temptations,My Girl,spotify:user:musicologo13:playlist:0000mSEZofZ...
5,1.164783,USRC16901355,Elvis Presley,Suspicious Minds,spotify:user:musicologo13:playlist:0000mSEZofZ...
6,1.164516,USLA19300013,Dion,Runaround Sue,spotify:user:musicologo13:playlist:0000mSEZofZ...
7,1.158101,USPR36105224,Bruce Channel,Hey! Baby,spotify:user:musicologo13:playlist:0000mSEZofZ...
8,1.156953,USSM19805299,Roy Orbison,"Oh, Pretty Woman",spotify:user:musicologo13:playlist:0000mSEZofZ...
9,1.153715,USCA20001587,The Beach Boys,Surfin' Safari - Remastered,spotify:user:musicologo13:playlist:0000mSEZofZ...


In [85]:
gbq.to_gbq(df_recs, 'adhoc.recs_for_will', project_id='umg-comm-tech-dev')

1it [00:04,  4.42s/it]


In [23]:
start_time = time.time()
classic_recs = model.recommend_outofmodel(classics, model.playlists_trainset, N=300)
rec_time = time.time() - start_time
print(rec_time)
recs = [(model.isrcs[x[0]],float(x[1]),x[2]) for x in classic_recs]
lookup_time = time.time() -start_time
print(lookup_time)

65.63360500335693
65.63467693328857


In [24]:
model.is

[('GBBPW0100148',
  1.1609696887546976,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBCEL1300087',
  1.1375965385346694,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBWZD0901801',
  1.125611789197092,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBWZD1002202',
  1.0837367034099454,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBWZD1103205',
  1.06041015898217,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBDCA0300279',
  1.0469497809628365,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBDDN0800296',
  1.0430579474128978,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GB5EM1401453',
  1.040994677372949,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBWZD1103702',
  1.0167044205658202,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBVTB1716001',
  1.0154142329486888,
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('DEN96163120

In [9]:
start_time = time.time()
classic_recs = model.recommend_outofmodel(classics, model.playlists_sparse, N=300)
rec_time = time.time() - start_time
print(rec_time)
recs_int = [(model.isrcs[x[0]],float(x[1]),x[2]) for x in classic_recs]
recs = [(x[0],x[1],lookup_dict['track_artist'][x[0]],
         lookup_dict['track_title'][x[0]], x[2]) for x in list(recs_int)]
lookup_dict_time = time.time() -start_time
print(lookup_dict_time)

KeyboardInterrupt: 

In [31]:
recs

[('GBBPW0100148',
  1.1609696887546976,
  'Aphex Twin',
  'Aphex Twin - Avril 14th',
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBCEL1300087',
  1.1375965385346694,
  'Jon Hopkins',
  'Abandon Window',
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBWZD0901801',
  1.125611789197092,
  'Nils Frahm',
  'Ambre',
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBWZD1002202',
  1.0837367034099454,
  'Ólafur Arnalds',
  'Þú ert jörðin',
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBWZD1103205',
  1.06041015898217,
  'A Winged Victory for the Sullen',
  'Steep Hills of Vicodin Tears',
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBDCA0300279',
  1.0469497809628365,
  'Max Richter',
  'On the Nature of Daylight',
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GBDDN0800296',
  1.0430579474128978,
  'Jon Hopkins',
  'Small Memory',
  'spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ'),
 ('GB5EM

In [51]:
keys = ['isrc','score','artist','title','seed_playlist_uri']
list_of_dict = [dict(zip(keys,x)) for x in recs]
json.dumps([dict(zip(keys,x)) for x in recs])

'[{"isrc": "GBBPW0100148", "score": 1.1609696887546976, "artist": "Aphex Twin", "title": "Aphex Twin - Avril 14th", "seed_playlist_uri": "spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ"}, {"isrc": "GBCEL1300087", "score": 1.1375965385346694, "artist": "Jon Hopkins", "title": "Abandon Window", "seed_playlist_uri": "spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ"}, {"isrc": "GBWZD0901801", "score": 1.125611789197092, "artist": "Nils Frahm", "title": "Ambre", "seed_playlist_uri": "spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ"}, {"isrc": "GBWZD1002202", "score": 1.0837367034099454, "artist": "\\u00d3lafur Arnalds", "title": "\\u00de\\u00fa ert j\\u00f6r\\u00f0in", "seed_playlist_uri": "spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ"}, {"isrc": "GBWZD1103205", "score": 1.06041015898217, "artist": "A Winged Victory for the Sullen", "title": "Steep Hills of Vicodin Tears", "seed_playlist_uri": "spotify:user:spotify:playlist:37i9dQZF1DX7XctuunSzQQ"}, {"isrc": "GBDCA03

In [52]:
with open('metadata_dict.pickle', 'wb') as f:
    pickle.dump(lookup_dict, f)

In [58]:
model.playlists_data.to_csv('model_20181112_playlists_data.csv', index=False)

In [59]:
gbq.to_gbq(model.playlists_data, project_id='umg-comm-tech-dev', destination_table='recommender.model_data', 
           if_exists='replace')

1it [01:49, 109.57s/it]


In [60]:
label_test = gbq.read_gbq(query='SELECT * FROM `umg-comm-tech-dev.recommender.track_metadata_augm`',
                         dialect='standard',project_id='umg-comm-tech-dev')

In [63]:
label_test.major_label.isnull().sum()

234562

In [64]:
label_test = label_test.fillna('Unknown')

In [65]:
label_test.describe(include='all')

,isrc,track_artist,track_title,major_label,label_studio,original_release_date,genre_name,parent_genre_name
count,1872604,1872604,1872604,1872604,1872604,1872604,1872604,1872604
unique,1872604,285642,1268415,5,148928,14500,332,49
top,USE831550870,Johann Sebastian Bach,Intro,Other,Unknown,Unknown,Unknown,Pop
freq,1,2632,863,1116763,234728,234562,238219,263751


In [66]:
gbq.to_gbq(label_test, destination_table='recommender.track_metadata_augm', project_id='umg-comm-tech-dev',
          if_exists='replace')

1it [00:15, 15.07s/it]


In [68]:
label_test = label_test.drop_duplicates(subset=['isrc'])
label_test.shape

(1872604, 8)

In [69]:
master_meta = label_test.set_index('isrc',drop=True)
master_meta_dict = master_meta.to_dict()

In [70]:
master_meta_dict

{'track_artist': {'CAW111700096': 'Matthew Good',
  'USGES1440001': 'Sleeping At Last',
  'AUUS00606436': 'Back In Black',
  'TCACP1698389': 'Anth',
  'TCADS1808368': 'Afterglow',
  'USLZJ1765444': 'Lucero',
  'BXZZO1700015': 'Lauana Prado',
  'MXF331112161': 'Miguel Mateos',
  'THMOB1301076': 'Big Ass',
  'GBLFP1234774': 'Kat Edmonson',
  'NLG270800619': 'La-33',
  'GBJ850300720': 'Dr Zeus, Master Rakesh, Shortie aka Little Lox & Deepti',
  'TWUM71200011': 'Karen Mok',
  'USHM91767430': 'Crater',
  'CH5850700001': 'Povia',
  'TR0200500267': 'İbrahim Tatlıses',
  'DEPL91800746': 'Johannes Albert',
  'GBFHB1768301': '2017 DJ Mo Fleeks',
  'COC011329508': 'Los Hermanos Lebron',
  'IT00C9800406': 'Alexia',
  'QM6MZ1439348': 'Yaira',
  'TCABH1265568': 'Nature Sounds',
  'NLG620605391': 'The London Film Score Orchestra',
  'TCABZ1431929': 'Ryan Hiraoka',
  'FR4KC1602013': 'Maurice Ravel',
  'FR6V81864061': 'Tony Camargo',
  'USQY51615249': 'Grupo Escolta',
  'GBHQC0553199': 'Tommy Dorsey',


In [71]:
with open('metadata_dict.pkl', 'wb') as f:
    pickle.dump(master_meta_dict, f)

In [5]:
with open('metadata_dict.pkl', 'rb') as f:
    meta_dict = pickle.load(f)

In [7]:
model_label = [(x,meta_dict['major_label'][x]) for x in list(model.isrcs)]

In [10]:
model_label_df = pd.DataFrame(model_label, columns=['isrc','major_label'])

In [11]:
model_label_df.head()

,isrc,major_label
0,TCACT1624704,Other
1,USUM71318966,UMG
2,USMC10400640,UMG
3,NLS5K1700040,Other
4,ARG511000031,UMG


In [12]:
len(model_label_df[model_label_df.major_label=='UMG'].isrc.unique())

248035

In [72]:
meta_data = pd.read_csv('/Users/daria/Downloads/recommender_metadata_metadata_20190205 (1).csv',
                        delimiter='\t', header=None, 
                        names = ['isrc','track_artist','track_title','major_label','label_studio',
                                 'original_release_date','genre_name','parent_genre_name'], dtype=str)

In [73]:
meta_data.head()

,isrc,track_artist,track_title,major_label,label_studio,original_release_date,genre_name,parent_genre_name
0,IL1058700005,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US7VG1629190,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USI4R1056103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USUYG1057294,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GBSMU0933322,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
len(meta_data.isrc.unique())

4273583

In [75]:
meta_data.describe()

,isrc,track_artist,track_title,major_label,label_studio,original_release_date,genre_name,parent_genre_name
count,4273582,3780296,3780301,3773696,3774295,3766724,3765251,3774295
unique,4273582,726784,2397835,340951,4,347,51,16680
top,GBSMU0932988,Various Artists,Intro,Columbia,Other,Pop,Pop,2013-01-01
freq,1,2596,1977,33195,2898084,454812,503297,16864


In [70]:
meta_data = meta_data.fillna('Unknown')
master_meta = meta_data.set_index('isrc', drop=True)

In [71]:
master_meta.head(100)

,track_artist,track_title,major_label,label_studio,original_release_date,genre_name,parent_genre_name
isrc,,,,,,,
USMSY0800240,Kelly Willard,Flow Through Me (Ode 12),Unknown,Unknown,Unknown,Unknown,Unknown
IT7680300001,LTJ XPerience,"Moon Beat, Pt. 2",Unknown,Unknown,Unknown,Unknown,Unknown
NOEIA1809010,"Paradise Hotel, Norway & Kristian René & Charl...",Nothing Lasts Forever,Unknown,Unknown,Unknown,Unknown,Unknown
RUA521203003,"Daniil Trifonov, Valery Gergiev & Mariinsky Or...","Piano Concerto No. 1 in B-Flat Minor, Op. 23: ...",Unknown,Unknown,Unknown,Unknown,Unknown
QM3971500002,Dopplebanger,Bang Bang (feat. Rude McClanahan & Spades),Unknown,Unknown,Unknown,Unknown,Unknown
UK2QP1800007,"Dream Scoring, Hear & Hakim Murphy",Blowed Out,Unknown,Unknown,Unknown,Unknown,Unknown
QZFCJ1800001,Popeye Caution & David Lyn,Work,Unknown,Unknown,Unknown,Unknown,Unknown
FIMAP9500018,Kaiho Aunesneva,Niiskun Lentohärveli,Unknown,Unknown,Unknown,Unknown,Unknown
GBRHF0710033,Joy Division,Shadowplay,Unknown,Unknown,Unknown,Unknown,Unknown


In [32]:
import requests
import json

In [49]:
isrcs = [ "GBEKZ0500052", "GBUM71503697", "DEF055901057", "DEN961731105", "US3M59908786", "GBF079811870", "DEB339866710" "GBF075340512", "DEF055731310", "US3M50009429", "AUAB01750561", "US3M50130717", "DEB330284407", "GBUM70903651", "DEF058705027", "US3M59897102", "GBUM70909602", "DEF055002524", "US3M51216908", "US3M50242121", "GBBBA7570814", "GBF079442808", "US3M59556506", "GBF077311643", "US3M50462108", "US3M50458504", "DEF058500141", "AUAB00350568", "ITUM70900146", "DEF058402506", "FRT150717001", "DEN960702022", "DEF058400513", "GBF077740106", "NLA509502614", "DEF056430107", "GBF078030724", "NLA507917302", "DEF056702662", "NLA505400128", "NLA509502661", "GBF071405582", "DEF059305584", "DEB331338413", "NLA509804305", "DEF059404043", "NLA508325605", "AUAB01350105", "GBF075441402", "NLA508807009", "NLA507400665", "NLA508202809", "DEF057930611", "DEN960201087", "DEB330915203", "GBF071540108", "US3M50242110", "GBA185730018", "AUAB01850113", "GBF076011380", "DEF056231940", "DEF055503105", "GBF071540111", "GBBBB9607511", "DEB331022109", "GBBBA0300712", "GBF077810973"]
df_isrc = pd.DataFrame(isrcs, columns=['isrcs'])
isrc_json = df_isrc.to_json()
r = requests.post('https://us-east1-umg-mir.cloudfunctions.net/reorder_isrcs/',json=isrc_json)

In [50]:
df_isrc = pd.DataFrame(isrcs, columns=['isrcs'])

In [51]:
isrc_json = df_isrc.to_json()

In [52]:
r = requests.post('https://us-east1-umg-mir.cloudfunctions.net/reorder_isrcs/',json=isrc_json)

In [53]:
r.text

'Error: could not handle the request\n'

In [42]:
isrcs[7:17]

['GBF079643039',
 'US3M59908755',
 'NLF7S1400001',
 'DEN960531208',
 'GBEKZ0500052',
 'GBUM71503697',
 'DEF055901057',
 'DEN961731105',
 'US3M59908786',
 'GBF079811870']